In [3]:
import psycopg2
from flask import Flask, jsonify, request

In [4]:
conn = psycopg2.connect("dbname=covid user=postgres password=postgres")
cur = conn.cursor()
cur.execute('select * from covid')
records = cur.fetchall()
records

[(1,
  'United States',
  datetime.date(2019, 12, 29),
  0,
  4111,
  434,
  4545,
  60170,
  98.0),
 (2,
  'United States',
  datetime.date(2020, 1, 5),
  1,
  4153,
  475,
  4628,
  60734,
  97.0),
 (3,
  'United States',
  datetime.date(2020, 1, 12),
  2,
  4066,
  468,
  4534,
  59362,
  98.0),
 (4,
  'United States',
  datetime.date(2020, 1, 19),
  3,
  3915,
  500,
  4418,
  59162,
  99.0),
 (5,
  'United States',
  datetime.date(2020, 1, 26),
  0,
  3818,
  481,
  4299,
  58834,
  99.0),
 (6,
  'United States',
  datetime.date(2020, 2, 2),
  4,
  3823,
  520,
  4346,
  59482,
  100.0),
 (7,
  'United States',
  datetime.date(2020, 2, 9),
  6,
  3848,
  561,
  4413,
  58815,
  100.0),
 (8,
  'United States',
  datetime.date(2020, 2, 16),
  6,
  3719,
  566,
  4288,
  58912,
  101.0),
 (9,
  'United States',
  datetime.date(2020, 2, 23),
  9,
  3842,
  657,
  4503,
  59334,
  103.0),
 (10,
  'United States',
  datetime.date(2020, 3, 1),
  38,
  3977,
  635,
  4630,
  59694,
  103.

In [5]:
output = []
for i in range(len(records)):
    temp_out = {
    'week' : records[i][0],
    'state' : records[i][1],
    'week_start' : records[i][2],
    'covid_d' : records[i][3],
    'pneum_d' : records[i][4],
    'flu_d' : records[i][5],
    'anyof3_d' : records[i][6],
    'total_d' : records[i][7],
    'poe_d' : records[i][8]}
    output.append(temp_out)
output


[{'week': 1,
  'state': 'United States',
  'week_start': datetime.date(2019, 12, 29),
  'covid_d': 0,
  'pneum_d': 4111,
  'flu_d': 434,
  'anyof3_d': 4545,
  'total_d': 60170,
  'poe_d': 98.0},
 {'week': 2,
  'state': 'United States',
  'week_start': datetime.date(2020, 1, 5),
  'covid_d': 1,
  'pneum_d': 4153,
  'flu_d': 475,
  'anyof3_d': 4628,
  'total_d': 60734,
  'poe_d': 97.0},
 {'week': 3,
  'state': 'United States',
  'week_start': datetime.date(2020, 1, 12),
  'covid_d': 2,
  'pneum_d': 4066,
  'flu_d': 468,
  'anyof3_d': 4534,
  'total_d': 59362,
  'poe_d': 98.0},
 {'week': 4,
  'state': 'United States',
  'week_start': datetime.date(2020, 1, 19),
  'covid_d': 3,
  'pneum_d': 3915,
  'flu_d': 500,
  'anyof3_d': 4418,
  'total_d': 59162,
  'poe_d': 99.0},
 {'week': 5,
  'state': 'United States',
  'week_start': datetime.date(2020, 1, 26),
  'covid_d': 0,
  'pneum_d': 3818,
  'flu_d': 481,
  'anyof3_d': 4299,
  'total_d': 58834,
  'poe_d': 99.0},
 {'week': 6,
  'state': 'Unite

In [6]:
def makeJSON(records):
    output = []
    for i in range(len(records)):
        temp_out = {
            'week' : records[i][0],
            'state' : records[i][1],
            'week_start' : records[i][2],
            'covid_d' : records[i][3],
            'pneum_d' : records[i][4],
            'flu_d' : records[i][5],
            'anyof3_d' : records[i][6],
            'total_d' : records[i][7],
            'poe_d' : records[i][8]}
        output.append(temp_out)
    return output

In [8]:
def pickState(state):
    sql = "select * from covid where state like (%s)"
    data = (state,)
    cur.execute(sql, data)
    records = cur.fetchall()
    output = makeJSON(records)
    return output

pickState("Illinois")


[{'week': 1,
  'state': 'Illinois',
  'week_start': datetime.date(2019, 12, 29),
  'covid_d': 0,
  'pneum_d': 144,
  'flu_d': None,
  'anyof3_d': 146,
  'total_d': 2191,
  'poe_d': 95.0},
 {'week': 2,
  'state': 'Illinois',
  'week_start': datetime.date(2020, 1, 5),
  'covid_d': 0,
  'pneum_d': 154,
  'flu_d': 10,
  'anyof3_d': 164,
  'total_d': 2252,
  'poe_d': 95.0},
 {'week': 3,
  'state': 'Illinois',
  'week_start': datetime.date(2020, 1, 12),
  'covid_d': 0,
  'pneum_d': 144,
  'flu_d': None,
  'anyof3_d': 153,
  'total_d': 2231,
  'poe_d': 97.0},
 {'week': 4,
  'state': 'Illinois',
  'week_start': datetime.date(2020, 1, 19),
  'covid_d': 0,
  'pneum_d': 157,
  'flu_d': 16,
  'anyof3_d': 173,
  'total_d': 2199,
  'poe_d': 98.0},
 {'week': 5,
  'state': 'Illinois',
  'week_start': datetime.date(2020, 1, 26),
  'covid_d': 0,
  'pneum_d': 155,
  'flu_d': 16,
  'anyof3_d': 171,
  'total_d': 2137,
  'poe_d': 97.0},
 {'week': 6,
  'state': 'Illinois',
  'week_start': datetime.date(2020,